In [1]:
import geopandas as gpd
import pandas as pd
pos_gdf = gpd.read_parquet("/home/christopher.x.ren/datasets/ra_tea/pos_gdf_v1_java_2024-11-08.parquet")
neg_ei_gdf = gpd.read_parquet("/home/christopher.x.ren/datasets/ra_tea/neg_gdf_v1_java_2024-11-08.parquet")
neg_lulc_gdf = gpd.read_parquet("/home/christopher.x.ren/datasets/ra_tea/java_neg_water_built_tree_rangeland_samples_10090.parquet").drop(columns=['remapped'], errors='ignore')
centroid_gdf = gpd.read_parquet("/home/christopher.x.ren/embeddings/ra_tea/centroid_gdf.parquet")
# Spatial join to get tile_ids for each point in neg_lulc_gdf
neg_lulc_with_tiles = gpd.sjoin_nearest(neg_lulc_gdf, centroid_gdf, how='left')

neg_ei_gdf['class'] = 'ei_neg'
neg_ei_gdf['label'] = 0
neg_lulc_with_tiles['label'] = 0

full_neg_gdf = pd.concat([neg_ei_gdf, neg_lulc_with_tiles.drop(columns=['index_right'])])
pos_gdf['class'] = 'ei_pos'
pos_gdf['label'] = 1
full_pos_gdf = pos_gdf

full_gdf = pd.concat([full_neg_gdf, full_pos_gdf]).drop_duplicates(subset=['tile_id'])
full_gdf.to_parquet("/home/christopher.x.ren/datasets/ra_tea/tile_classifier_dataset_v1_java_2024-11-10.parquet")

/home/christopher.x.ren/miniforge3/envs/ei-nb-v2/lib/python3.12/site-packages/geopandas/array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [2]:
import duckdb
conn = duckdb.connect('/home/christopher.x.ren/embeddings/ra_tea/embeddings.db')
tile_ids = tuple(full_gdf.tile_id.unique())
embeddings_df = conn.execute(f"""
    SELECT *
    FROM embeddings 
    WHERE tile_id IN {tile_ids}
""").df()
embeddings_df
full_df_embeddings = gpd.GeoDataFrame(embeddings_df.merge(full_gdf[['tile_id', 'label', 'geometry', 'class']], on='tile_id', how='inner'), geometry='geometry')
full_df_embeddings = full_df_embeddings.drop(columns=['row_number'])
full_df_embeddings.to_parquet("/home/christopher.x.ren/datasets/ra_tea/tile_classifier_dataset_v1_java_2024-11-10_embeddings.parquet")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))